In [1]:
# https://github.com/shenweichen/DeepMatch
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set,gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

In [2]:
# def main():
data = pd.read_csv("./movielens_sample.txt")
data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
228,3,3671,5,978297419,Blazing Saddles (1974),Comedy|Western,M,25,15,55117
229,3,2871,4,978297539,Deliverance (1972),Adventure|Thriller,M,25,15,55117
230,3,2115,4,978297777,Indiana Jones and the Temple of Doom (1984),Action|Adventure,M,25,15,55117
231,3,1136,5,978298079,Monty Python and the Holy Grail (1974),Comedy,M,25,15,55117


In [3]:
q1= np.arr

0.7071067811865475